### Задание 1

In [8]:
import pandas as pd
import zipfile
import json
import sqlite3
from pathlib import Path

In [2]:
# Читаем файл json и сохраняем в DataFrame
okved_df = pd.read_json('okved_2.json.zip')

In [3]:
okved_df

,code,parent_code,section,name,comment
0,01,A,A,"Растениеводство и животноводство, охота и пред...",Эта группировка включает:\n- два основных вида...
1,01.1,01,A,Выращивание однолетних культур,Эта группировка включает:\n- выращивание однол...
2,01.11,01.1,A,"Выращивание зерновых (кроме риса), зернобобовы...",Эта группировка включает:\n- все формы выращив...
3,01.11.1,01.11,A,Выращивание зерновых культур,
4,01.11.11,01.11.1,A,Выращивание пшеницы,
...,...,...,...,...,...
2813,Q,,Q,ДЕЯТЕЛЬНОСТЬ В ОБЛАСТИ ЗДРАВООХРАНЕНИЯ И СОЦИА...,Этот раздел включает:\n- предоставление деятел...
2814,R,,R,"ДЕЯТЕЛЬНОСТЬ В ОБЛАСТИ КУЛЬТУРЫ, СПОРТА, ОРГАН...",Данный раздел включает:\n- широкий спектр видо...
2815,S,,S,ПРЕДОСТАВЛЕНИЕ ПРОЧИХ ВИДОВ УСЛУГ,Этот раздел включает:\n- деятельность обществе...
2816,T,,T,ДЕЯТЕЛЬНОСТЬ ДОМАШНИХ ХОЗЯЙСТВ КАК РАБОТОДАТЕЛ...,


In [4]:
okved_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2818 entries, 0 to 2817
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   code         2818 non-null   object
 1   parent_code  2818 non-null   object
 2   section      2818 non-null   object
 3   name         2818 non-null   object
 4   comment      2818 non-null   object
dtypes: object(5)
memory usage: 110.2+ KB


In [5]:
# Создаём БД hw1.db и таблицу okved в ней
create_stmt = \
"""
create table if not exists okved(
    code text,
    parent_code text,
    section text,
    name text,
    comment text
)
"""
with sqlite3.connect('hw1.db') as connection:
    cursor = connection.cursor()
    try:
        cursor.execute(create_stmt)
    except(Exception) as error:
        print(error)
        connection.rollback()
    else:
        connection.commit()
    finally:
        cursor.close()

In [6]:
# Записываем данные из DataFrame в БД
with sqlite3.connect('hw1.db') as connection:
    okved_df.to_sql('okved', connection, if_exists='append', index=False, method='multi')

In [7]:
# Проверяем получившийся результат
with sqlite3.connect('hw1.db') as connection:
    df = pd.read_sql('select * from okved', connection)

display(df)

,code,parent_code,section,name,comment
0,01,A,A,"Растениеводство и животноводство, охота и пред...",Эта группировка включает:\n- два основных вида...
1,01.1,01,A,Выращивание однолетних культур,Эта группировка включает:\n- выращивание однол...
2,01.11,01.1,A,"Выращивание зерновых (кроме риса), зернобобовы...",Эта группировка включает:\n- все формы выращив...
3,01.11.1,01.11,A,Выращивание зерновых культур,
4,01.11.11,01.11.1,A,Выращивание пшеницы,
...,...,...,...,...,...
2813,Q,,Q,ДЕЯТЕЛЬНОСТЬ В ОБЛАСТИ ЗДРАВООХРАНЕНИЯ И СОЦИА...,Этот раздел включает:\n- предоставление деятел...
2814,R,,R,"ДЕЯТЕЛЬНОСТЬ В ОБЛАСТИ КУЛЬТУРЫ, СПОРТА, ОРГАН...",Данный раздел включает:\n- широкий спектр видо...
2815,S,,S,ПРЕДОСТАВЛЕНИЕ ПРОЧИХ ВИДОВ УСЛУГ,Этот раздел включает:\n- деятельность обществе...
2816,T,,T,ДЕЯТЕЛЬНОСТЬ ДОМАШНИХ ХОЗЯЙСТВ КАК РАБОТОДАТЕЛ...,


In [10]:
df[df.name.str.lower() == 'деятельность в сфере телекоммуникаций']

,code,parent_code,section,name,comment
2193,61,J,J,Деятельность в сфере телекоммуникаций,Эта группировка включает:\n- деятельность по п...


### Задание 2

In [2]:
import pandas as pd
import zipfile
import sqlite3
from pathlib import Path
from tqdm import tqdm
import os

In [17]:
file_path = Path('C:', 'Users', 'greym', 'Downloads', 'hw1', 'egrul.json.zip')
NPROC = os.cpu_count() - 1  # количество процессоров
MAX_WORKERS = min(32, os.cpu_count() + 4)  # количество потоков


def unpacker(path: 'str | Path') -> pd.DataFrame:
    """Распаковщик данных из zip-архива и парсер JSON"""

    def parser_json(json_file: str) -> pd.DataFrame:
        """Парсит один файл JSON"""
        with zipobj.open(json_file) as file:
            return pd.read_json(file)

    with zipfile.ZipFile(path, 'r') as zipobj:
        files = zipobj.namelist()
        for file in files:
            yield parser_json(file)


def parser_data(value: dict) -> pd.DataFrame:
    """Парсер атрибута data"""
    # Получаем значения КодОКВЭД и НаимОКВЭД. Если нет, устанавливаем значения в None
    main_val = dict(code_okved=value.get('СвОКВЭД', {}).get('СвОКВЭДОсн', {}).get('КодОКВЭД'),
                    name_okved=value.get('СвОКВЭД', {}).get('СвОКВЭДОсн', {}).get('НаимОКВЭД'),
                    type_okved='Осн')
    # Фильтруем по 61 ОКВЭД
    code = main_val['code_okved']
    return main_val if code and code.startswith('61') else None


def processor_df(df: pd.DataFrame) -> pd.DataFrame:
    """Парсер и обработчик данных"""
    # Парсим атрибут data и записываем данные в общий DF
    df = df.assign(data=df.data.apply(parser_data))
    # Исключаем пустые данные (т.е. те, что не попали под ОКВЭД 61)
    df = df[df.data.notna()]
    # Разбиваем атрибут data на столбцы, и добавляем их в общий DF
    cols = ['ogrn', 'inn', 'kpp', 'name', 'full_name']
    return df[cols].join(df.data.apply(pd.Series))


# Создаём генератор-парсер данных из .zip архива в формате JSON
gen_dfs = unpacker(file_path)

# Парсим данные из JSON файлов в DF в параллельном режиме
dfs = map(processor_df, gen_dfs)

In [ ]:
# Объединяем все данные по всем юр. лицам в один DF
egrul_df = pd.concat(tqdm(dfs), ignore_index=True)

In [34]:
# Удаляем лишний столбец data
egrul_df.drop(columns='data', inplace=True)

In [36]:
# Создаём таблицу telecom_companies БД hw1.db
create_stmt = \
"""
create table if not exists telecom_companiesokved(
    inn integer,
    ogrn integer,
    kpp integer,
    name text,
    full_name text,
    code_okved text,
    name_okved text,
    type_okved text
)
"""
with sqlite3.connect('hw1.db') as connection:
    cursor = connection.cursor()
    try:
        cursor.execute(create_stmt)
    except(Exception) as error:
        print(error)
        connection.rollback()
    else:
        connection.commit()
    finally:
        cursor.close()

In [37]:
# Записываем данные из DataFrame в БД
with sqlite3.connect('hw1.db') as connection:
    egrul_df.to_sql('telecom_companiesokved', connection,
                    if_exists='append',
                    index=False,
                    method='multi')

In [4]:
# Проверяем получившийся результат
with sqlite3.connect('hw1.db') as connection:
    df = pd.read_sql('select distinct * from telecom_companiesokved', connection)

display(df)

,inn,ogrn,kpp,name,full_name,code_okved,name_okved,type_okved
0,7536178239,1197536005787,753601001,"ООО ""ИНТЕР КОД""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ИНТЕ...",61.90,Деятельность в области телекоммуникаций прочая,Осн
1,7513007283,1197536005919,751301001,"ООО ""ВОНГОЛА""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ВОНГ...",61.10,Деятельность в области связи на базе проводных...,Осн
2,7604352760,1197627000691,760401001,"ООО ""РЭС""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""РИНГ...",61.10,Деятельность в области связи на базе проводных...,Осн
3,7811531478,1127847488053,781101001,"ООО ""СМО СИСТЕМА""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""СМО ...",61.30,Деятельность в области спутниковой связи,Осн
4,7714772929,1097746001143,773401001,"ЗАО ""ИНТЕРФОН""","ЗАКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""ИНТЕРФОН""",61.10,Деятельность в области связи на базе проводных...,Осн
...,...,...,...,...,...,...,...,...
14555,274073307,1020202562490,27401001,"ООО ""ВТКОМ""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ВОЛН...",61.10,Деятельность в области связи на базе проводных...,Осн
14556,275039130,1020202764548,164801001,"ООО ""ЕС ТЕЛЕКОМ""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЕДИН...",61.10.4,Деятельность в области документальной электрос...,Осн
14557,6455056632,1126455002145,645201001,"ООО ""ХАЙНЦЕЛЬМЕНХЕН""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ХАЙН...",61.10,Деятельность в области связи на базе проводных...,Осн
14558,6501245933,1126501002275,650101001,"ООО ""ИНТУИЦИЯ ТЕЛЕКОМ""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ИНТУ...",61.10,Деятельность в области связи на базе проводных...,Осн


In [60]:
with sqlite3.connect('hw1.db') as connection:
    stmt = \
    """
    create table tmp as select distinct * from telecom_companiesokved;
    delete from telecom_companiesokved;
    insert into telecom_companiesokved select * from tmp;
    drop table if exists tmp;
    """
    connection.executescript(stmt)